In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

#packages required for image preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.metrics import mean_absolute_error
import tensorflow as tf

#loading training and testing datasets
df_train = pd.read_csv('../input/rsna-bone-age/boneage-training-dataset.csv')
df_test = pd.read_csv('../input/rsna-bone-age/boneage-test-dataset.csv')

#appending png file extension to id column for both training and testing datasets
df_train['id'] = df_train['id'].apply(lambda x: str(x)+'.png')
df_test['Case ID'] = df_test['Case ID'].apply(lambda x: str(x)+'.png')

#Feature Engineering
df_train['Sex'] = df_train['male'].apply(lambda x: '1' if x else '0')
del(df_train['male'])
df_test['id'] = df_test['Case ID']
del(df_test['Case ID'])

#splitting train datasets into traininng and validation datasets
train_df, test_df = train_test_split(df_train, test_size = 0.20, random_state = 0)
#train_df, test_df = train_test_split(test_train_df, test_size = 0.05, random_state = 0)

In [2]:
test_df.info()
print(len(train_df))
print(len(test_df))
#print(len(valid_df))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2523 entries, 5112 to 7227
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2523 non-null   object
 1   boneage  2523 non-null   int64 
 2   Sex      2523 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.8+ KB
10088
2523


In [3]:
GT_train = train_df['boneage']
GT_train = np.asarray(GT_train.tolist())
GT_test = test_df['boneage']
GT_test = np.asarray(GT_test.tolist())

In [4]:
print(len(GT_train))
print(len(GT_test))

10088
2523


In [5]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, directory, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=False):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        self.n_sex = df[X_col['Sex']].nunique()
        #self.n_path = df[X_col['id']]
        self.directory = directory
        #self.GT = []
        
    
    def __get_input(self, path, target_size):
        img_path = self.directory+path
        #print(img_path)
        #self.id.append(path)
        image = tf.keras.preprocessing.image.load_img(img_path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy()

        return image_arr/255.
    
    def __get_gender(self, label, num_classes):
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
        
        
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['id']]
        sex_batch = batches[self.X_col['Sex']]

        bone_batch = batches[self.y_col['boneage']]

        X0_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])
        X1_batch = np.asarray([self.__get_gender(y, self.n_sex) for y in sex_batch])
        y_batch = np.asarray(bone_batch)
        #self.GT.append(y_batch)
        

        return tuple([X0_batch,X1_batch]), y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __getitem__(self, index):

        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [6]:
def create_model(pretrain_model):
    
    x = pretrain_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    output_image = Dense(1,activation='linear')(x)

    #image_output = keras.Model(input=pretrain_model, output=output_image)
    gender_input = keras.Input(shape=(2,),name = 'gender')
    gender_concat =layers.concatenate([output_image,gender_input])
    output_gen = Dense(1,activation='linear')(gender_concat)
    #gender_model = keras.Model(inputs = gender_input,outputs = output_gen)


    model = keras.Model(inputs =[pretrain_model.input, gender_input],outputs=[output_gen])

    return model
    

In [7]:
def create_generator(dataframe,image_size,path_directory,bbatch_size):
    target_size = (image_size, image_size, 3)
    
    generator = CustomDataGen(dataframe,
                             directory=path_directory,
                             X_col={'id':'id','Sex': 'Sex'},
                             y_col={'boneage': 'boneage'},
                             batch_size=bbatch_size, input_size=target_size)
    return generator

In [8]:
from keras.applications import DenseNet121 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
DenseNet121_model = create_model(pretrain_model)
DenseNet121_model.load_weights('../input/bone-age-detection-all-weights/DenseNet121_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
DenseNet121_testgen = create_generator(test_df,image_size,direc,bbatch_size)
DenseNet121_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


DenseNet121_test = np.asarray(DenseNet121_model.predict(DenseNet121_testgen)).flatten()
print('Test')
print(len(DenseNet121_test))
DenseNet121_train = np.asarray(DenseNet121_model.predict(DenseNet121_traingen)).flatten()
print('Train')
print(len(DenseNet121_train))

29089792/29084464 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [9]:
from keras.applications import DenseNet169 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
DenseNet169_model = create_model(pretrain_model)
DenseNet169_model.load_weights('../input/bone-age-detection-all-weights/DenseNet169_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
DenseNet169_testgen = create_generator(test_df,image_size,direc,bbatch_size)
DenseNet169_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


DenseNet169_test = np.asarray(DenseNet169_model.predict(DenseNet169_testgen)).flatten()
print('Test')
print(len(DenseNet169_test))
DenseNet169_train = np.asarray(DenseNet169_model.predict(DenseNet169_traingen)).flatten()
print('Train')
print(len(DenseNet169_train))

51879936/51877672 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [10]:
from keras.applications import DenseNet201 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
DenseNet201_model = create_model(pretrain_model)
DenseNet201_model.load_weights('../input/bone-age-detection-all-weights/DenseNet201_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
DenseNet201_testgen = create_generator(test_df,image_size,direc,bbatch_size)
DenseNet201_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


DenseNet201_test = np.asarray(DenseNet201_model.predict(DenseNet201_testgen)).flatten()
print('Test')
print(len(DenseNet201_test))
DenseNet201_train = np.asarray(DenseNet201_model.predict(DenseNet201_traingen)).flatten()
print('Train')
print(len(DenseNet201_train))

74842112/74836368 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [11]:
from keras.applications import InceptionResNetV2 as pre_model
image_size =299
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
InceptionResNetV2_model = create_model(pretrain_model)
InceptionResNetV2_model.load_weights('../input/bone-age-detection-all-weights/InceptionResNet_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
InceptionResNetV2_testgen = create_generator(test_df,image_size,direc,bbatch_size)
InceptionResNetV2_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


InceptionResNetV2_test = np.asarray(InceptionResNetV2_model.predict(InceptionResNetV2_testgen)).flatten()
print('Test')
print(len(InceptionResNetV2_test))
InceptionResNetV2_train = np.asarray(InceptionResNetV2_model.predict(InceptionResNetV2_traingen)).flatten()
print('Train')
print(len(InceptionResNetV2_train))

219062272/219055592 [==============================] - 2s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [12]:
from keras.applications import InceptionV3 as pre_model
image_size =299
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
InceptionV3_model = create_model(pretrain_model)
InceptionV3_model.load_weights('../input/bone-age-detection-all-weights/InceptionV3_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
InceptionV3_testgen = create_generator(test_df,image_size,direc,bbatch_size)
InceptionV3_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


InceptionV3_test = np.asarray(InceptionV3_model.predict(InceptionV3_testgen)).flatten()
print('Test')
print(len(InceptionV3_test))
InceptionV3_train = np.asarray(InceptionV3_model.predict(InceptionV3_traingen)).flatten()
print('Train')
print(len(InceptionV3_train))

87916544/87910968 [==============================] - 2s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [13]:
from keras.applications import MobileNetV2 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
MobileNetV2_model = create_model(pretrain_model)
MobileNetV2_model.load_weights('../input/bone-age-detection-all-weights/MobileNetV2_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
MobileNetV2_testgen = create_generator(test_df,image_size,direc,bbatch_size)
MobileNetV2_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


MobileNetV2_test = np.asarray(MobileNetV2_model.predict(MobileNetV2_testgen)).flatten()
print('Test')
print(len(MobileNetV2_test))
MobileNetV2_train = np.asarray(MobileNetV2_model.predict(MobileNetV2_traingen)).flatten()
print('Train')
print(len(MobileNetV2_train))

9412608/9406464 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [14]:
from keras.applications import MobileNet as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
MobileNet_model = create_model(pretrain_model)
MobileNet_model.load_weights('../input/bone-age-detection-all-weights/MobileNet_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
MobileNet_testgen = create_generator(test_df,image_size,direc,bbatch_size)
MobileNet_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


MobileNet_test = np.asarray(MobileNet_model.predict(MobileNet_testgen)).flatten()
print('Test')
print(len(MobileNet_test))
MobileNet_train = np.asarray(MobileNet_model.predict(MobileNet_traingen)).flatten()
print('Train')
print(len(MobileNet_train))

17227776/17225924 [==============================] - 1s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [15]:
from keras.applications import NASNetLarge as pre_model
image_size =331
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
NASNetLarge_model = create_model(pretrain_model)
NASNetLarge_model.load_weights('../input/bone-age-detection-all-weights/NasNetLarge_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
NASNetLarge_testgen = create_generator(test_df,image_size,direc,bbatch_size)
NASNetLarge_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


NASNetLarge_test = np.asarray(NASNetLarge_model.predict(NASNetLarge_testgen)).flatten()
print('Test')
print(len(NASNetLarge_test))
NASNetLarge_train = np.asarray(NASNetLarge_model.predict(NASNetLarge_traingen)).flatten()
print('Train')
print(len(NASNetLarge_train))

343613440/343610240 [==============================] - 6s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [16]:
from keras.applications import NASNetMobile as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
NASNetMobile_model = create_model(pretrain_model)
NASNetMobile_model.load_weights('../input/bone-age-detection-all-weights/NasNetMobile_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
NASNetMobile_testgen = create_generator(test_df,image_size,direc,bbatch_size)
NASNetMobile_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


NASNetMobile_test = np.asarray(NASNetMobile_model.predict(NASNetMobile_testgen)).flatten()
print('Test')
print(len(NASNetMobile_test))
NASNetMobile_train = np.asarray(NASNetMobile_model.predict(NASNetMobile_traingen)).flatten()
print('Train')
print(len(NASNetMobile_train))

19996672/19993432 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [17]:
from keras.applications import ResNet101V2 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
ResNet101V2_model = create_model(pretrain_model)
ResNet101V2_model.load_weights('../input/bone-age-detection-all-weights/ResNet101V2_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
ResNet101V2_testgen = create_generator(test_df,image_size,direc,bbatch_size)
ResNet101V2_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


ResNet101V2_test = np.asarray(ResNet101V2_model.predict(ResNet101V2_testgen)).flatten()
print('Test')
print(len(ResNet101V2_test))
ResNet101V2_train = np.asarray(ResNet101V2_model.predict(ResNet101V2_traingen)).flatten()
print('Train')
print(len(ResNet101V2_train))

171319296/171317808 [==============================] - 4s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [18]:
from keras.applications import ResNet152V2 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
ResNet152V2_model = create_model(pretrain_model)
ResNet152V2_model.load_weights('../input/bone-age-detection-all-weights/ResNet152V2_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
ResNet152V2_testgen = create_generator(test_df,image_size,direc,bbatch_size)
ResNet152V2_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


ResNet152V2_test = np.asarray(ResNet152V2_model.predict(ResNet152V2_testgen)).flatten()
print('Test')
print(len(ResNet152V2_test))
ResNet152V2_train = np.asarray(ResNet152V2_model.predict(ResNet152V2_traingen)).flatten()
print('Train')
print(len(ResNet152V2_train))

234553344/234545216 [==============================] - 3s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [19]:
from keras.applications import VGG16 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
VGG16_model = create_model(pretrain_model)
VGG16_model.load_weights('../input/bone-age-detection-all-weights/VGG-16_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
VGG16_testgen = create_generator(test_df,image_size,direc,bbatch_size)
VGG16_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


VGG16_test = np.asarray(VGG16_model.predict(VGG16_testgen)).flatten()
print('Test')
print(len(VGG16_test))
VGG16_train = np.asarray(VGG16_model.predict(VGG16_traingen)).flatten()
print('Train')
print(len(VGG16_train))

58892288/58889256 [==============================] - 0s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [20]:
from keras.applications import VGG19 as pre_model
image_size =224
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
VGG19_model = create_model(pretrain_model)
VGG19_model.load_weights('../input/bone-age-detection-all-weights/VGG-19_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
VGG19_testgen = create_generator(test_df,image_size,direc,bbatch_size)
VGG19_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


VGG19_test = np.asarray(VGG19_model.predict(VGG19_testgen)).flatten()
print('Test')
print(len(VGG19_test))
VGG19_train = np.asarray(VGG19_model.predict(VGG19_traingen)).flatten()
print('Train')
print(len(VGG19_train))

80142336/80134624 [==============================] - 1s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [21]:
from keras.applications import Xception as pre_model
image_size =299
pretrain_model = pre_model(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
Xception_model = create_model(pretrain_model)
Xception_model.load_weights('../input/bone-age-detection-all-weights/Xception_weights.h5')
print('Model Load Done')


bbatch_size = 1
direc = '../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/'
Xception_testgen = create_generator(test_df,image_size,direc,bbatch_size)
Xception_traingen = create_generator(train_df,image_size,direc,bbatch_size)
print('Generator Created')
print('Start Training...')


Xception_test = np.asarray(Xception_model.predict(Xception_testgen)).flatten()
print('Test')
print(len(Xception_test))
Xception_train = np.asarray(Xception_model.predict(Xception_traingen)).flatten()
print('Train')
print(len(Xception_train))

83689472/83683744 [==============================] - 1s 0us/step
Model Load Done
Generator Created
Start Training...
Test
2523
Train
10088


In [22]:
train_data = {
    'GT' : GT_train,
    'DenseNet121' : DenseNet121_train,
    'DenseNet169' : DenseNet169_train,
    'DenseNet201' : DenseNet201_train,
    'InceptionResNetV2' : InceptionResNetV2_train,
    'InceptionV3' : InceptionV3_train,
    'MobileNetV2' : MobileNetV2_train,
    'MobileNet' : MobileNet_train,
    'NASNetLarge' : NASNetLarge_train,
    'NASNetMobile' : NASNetMobile_train,
    'ResNet101V2' : ResNet101V2_train,
    'ResNet152V2' : ResNet152V2_train,
    'VGG16' : VGG16_train,
    'VGG19' : VGG19_train,
    'Xception' : Xception_train
    
}

train_dataframe = pd.DataFrame(train_data,columns = ['GT','DenseNet121','DenseNet169','DenseNet201','InceptionResNetV2','InceptionV3','MobileNetV2','MobileNet','NASNetLarge','NASNetMobile','ResNet101V2','ResNet152V2','VGG16','VGG19','Xception'])
print(train_dataframe.head())
print(train_dataframe.tail())
train_dataframe.to_csv('Fire_train.csv')

    GT  DenseNet121  DenseNet169  DenseNet201  InceptionResNetV2  InceptionV3  \
0  132   124.704643   147.041565   141.995026         136.747604   137.403656   
1  168   163.556061   159.781235   159.573807         167.169601   169.998459   
2  162   143.877441   144.899078   149.350800         159.044571   161.347260   
3  162   170.720093   159.217590   163.647507         166.479462   161.176208   
4  168   166.893616   173.117661   158.186142         162.785767   170.292709   

   MobileNetV2   MobileNet  NASNetLarge  NASNetMobile  ResNet101V2  \
0   125.635910  133.086380   130.202728    135.204132   137.127441   
1   160.012207  170.389648   165.332092    168.352402   165.000000   
2   163.003220  163.888397   157.763977    162.643616   151.092621   
3   162.123520  163.971222   155.176544    158.054688   160.031372   
4   159.768723  166.949692   163.965088    166.140610   170.354828   

   ResNet152V2       VGG16       VGG19    Xception  
0   131.821457  143.008057  119.155495 

In [23]:
test_data = {
    'GT' : GT_test,
    'DenseNet121' : DenseNet121_test,
    'DenseNet169' : DenseNet169_test,
    'DenseNet201' : DenseNet201_test,
    'InceptionResNetV2' : InceptionResNetV2_test,
    'InceptionV3' : InceptionV3_test,
    'MobileNetV2' : MobileNetV2_test,
    'MobileNet' : MobileNet_test,
    'NASNetLarge' : NASNetLarge_test,
    'NASNetMobile' : NASNetMobile_test,
    'ResNet101V2' : ResNet101V2_test,
    'ResNet152V2' : ResNet152V2_test,
    'VGG16' : VGG16_test,
    'VGG19' : VGG19_test,
    'Xception' : Xception_test
    
}

test_dataframe = pd.DataFrame(test_data,columns = ['GT','DenseNet121','DenseNet169','DenseNet201','InceptionResNetV2','InceptionV3','MobileNetV2','MobileNet','NASNetLarge','NASNetMobile','ResNet101V2','ResNet152V2','VGG16','VGG19','Xception'])
print(test_dataframe.head())
print(test_dataframe.tail())
test_dataframe.to_csv('Fire_train.csv')

    GT  DenseNet121  DenseNet169  DenseNet201  InceptionResNetV2  InceptionV3  \
0   94    92.287689   112.896729   113.184540         100.027306   103.150352   
1  174   174.616806   164.017929   174.928940         164.573715   167.706436   
2  144   154.213806   142.910889   151.876099         161.770416   153.149170   
3  168   170.734299   165.309418   171.187256         154.338562   164.622681   
4  108   105.397675   105.510628   121.320709         131.033295   131.861481   

   MobileNetV2   MobileNet  NASNetLarge  NASNetMobile  ResNet101V2  \
0   105.896248  105.699486   101.972626    102.021889   109.276474   
1   169.165421  181.361771   172.389359    176.266571   167.355789   
2   135.921326  149.306473   147.038757    148.637070   151.930069   
3   153.340729  159.576599   159.371750    160.950211   159.165848   
4   130.740250  115.557671   134.768021    118.084450   136.879211   

   ResNet152V2       VGG16       VGG19    Xception  
0   135.615265  131.287338  106.251366 